This notebook contains examples of using model index and OPC UA functions to download inverters, strings set and trackers data of the site 'EG-AS' from the model index and OPC UA api servers. The aggregated historical data downloaded in the data folder and cell execution time can be noted for each request.

### Import Libraries

In [ ]:
%load_ext autotime

In [1]:
# Import the required packeages
import pandas as pd
from typing import List, Dict
import sys
import os
import numpy as np
import json
import asyncio

### Import Scripts

In [2]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/andreaslydersen/Documents/Workspace/prediktor/pyPrediktorMapClient/src/pyprediktormapclient


In [3]:
# Import model index functions
from model_index import ModelIndex

# Import OPC UA functions
from opc_ua import OPC_UA

In [4]:
# Connection to the servers
model_index_url = "http://10.241.68.86:7001/v1/"
opcua_rest_url = "http://10.241.68.86:13371/"
opcua_server_url = "opc.tcp://10.241.80.4:4872"

# Model index API
mdx = ModelIndex(url=model_index_url)

# OPC UA API
opc = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url)

### OPC UA server parameters

In [5]:
# Input parameters for value data

# Parameters for aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z"  
end_time = "2022-07-06T07:55:14.544000Z" # 5th July aggregated data
end_time1 = "2022-07-20T07:55:14.544000Z" # 15 days
end_time2 = "2022-08-05T07:55:14.544000Z" # 30 days
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

In [6]:
# All the sites on the OPC server
sites = mdx.get_objects_of_type('SiteType', return_format="dataframe")
sites

TypeError: ModelIndex.get_objects_of_type() got an unexpected keyword argument 'return_type'

In [7]:
# Selecting specific site 
sitename = 'EG-AS'
site = sites[sites['DisplayName'] == sitename]

INVERTERS DATA

In [8]:
# All the inverters on the site
inverters = mdx.as_dataframe(mdx.get_object_descendants("InverterType", site, "PV_Assets"))
inverters

,ObjectId,DescendantId,DescendantName,DescendantType,ObjectName,Vars,Props
0,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19,EG-AS-TS10-I19,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
1,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I20,EG-AS-TS10-I20,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
2,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS11.I21,EG-AS-TS11-I21,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
3,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS11.I22,EG-AS-TS11-I22,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
4,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z2.TS09.I17,EG-AS-TS09-I17,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
5,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z2.TS09.I18,EG-AS-TS09-I18,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
6,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS03.I05,EG-AS-TS03-I05,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
7,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS03.I06,EG-AS-TS03-I06,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
8,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS06.I11,EG-AS-TS06-I11,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
9,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS06.I12,EG-AS-TS06-I12,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."


In [9]:
# List of unique inverter variables
inv_var_list = ['ACActivePower', 'ACCurrentAverage', 'ACCurrentPhaseA', 'ACCurrentPhaseB', 'ACCurrentPhaseC', 'ACDailyActiveEnergy', 'ACReactivePower', 'ACTotalActiveEnergy', 
'ACVoltagePhaseAB', 'ACVoltagePhaseAC', 'ACVoltagePhaseBC', 'DCCurrent', 'DCPower', 'DCVoltage', 'Frequency', 'PowerFactor', 'Status']

In [10]:
# Live value inverters data
inv_liv_values = opc.get_live_values_data(inv_var_list, inverters)
inv_liv_values

,Id,Name,Variable,Timestamp,Value,Code,Quality
0,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACActiveP...,EG-AS-TS10-I19,ACActivePower,2022-09-01T11:41:30.3706902Z,2285500.0,NaN,NaN
1,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentAverage,2022-09-01T11:41:30.3706902Z,2006.7,NaN,NaN
2,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentPhaseA,2022-09-01T11:41:30.3706902Z,2008.8,NaN,NaN
3,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentPhaseB,2022-09-01T11:41:30.3706902Z,2006.8,NaN,NaN
4,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentPhaseC,2022-09-01T11:41:30.3706902Z,2004.5,NaN,NaN
...,...,...,...,...,...,...,...
369,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.DCPower,EG-AS-TS05-I10,DCPower,2022-09-01T11:41:30.3706902Z,2136000.0,NaN,NaN
370,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.DCVoltage,EG-AS-TS05-I10,DCVoltage,2022-09-01T11:41:30.3706902Z,1022.9,NaN,NaN
371,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.Frequency,EG-AS-TS05-I10,Frequency,2022-04-27T10:17:52.428Z,0.0,2.147484e+09,Bad
372,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.PowerFactor,EG-AS-TS05-I10,PowerFactor,2022-04-27T10:17:52.428Z,0.0,2.147484e+09,Bad


In [11]:
# 1 day aggregated historical inverter data
await opc.get_agg_hist_value_data(start_time=start_time, end_time=end_time, pro_interval=pro_interval, agg_name=agg_name, obj_dataframe=inverters, include_variables=inv_var_list)

In [14]:
# 15 day inverters data
await opc.get_agg_hist_value_data(start_time, end_time1, pro_interval, agg_name, inverters, inv_var_list)

In [15]:
# 1 month inverter data 
await opc.get_agg_hist_value_data(start_time, end_time2, pro_interval, agg_name, inverters, inv_var_list)

STRINGS DATA

In [ ]:
# Strings set data 
strings = mdx.as_dataframe(mdx.get_object_descendants("StringSetType", site, "PV_Assets"))
strings

In [17]:
# List of strings variables
strng_var_list = ['DCCurrent', 'DCPower', 'DCVoltage', 'RelativeDCPower']

In [ ]:
# Live value strings data 
strng_liv_values = opc.get_live_values_data(strng_var_list, strings)
strng_liv_values

In [20]:
# 1 day strings data
await opc.get_agg_hist_value_data(start_time, end_time, pro_interval, agg_name, strings, strng_var_list)

In [21]:
# 15 days strings data
await opc.get_agg_hist_value_data(start_time, end_time1, pro_interval, agg_name, strings, strng_var_list)

In [ ]:
# 1 month strings data
await opc.get_agg_hist_value_data(start_time, end_time2, pro_interval, agg_name, strings, strng_var_list)

TRACKERS DATA

In [5]:
# Trackers data 
trackers = mdx.get_object_ancestors_as_dataframe("TrackerType", strings, "PV_Serves")
trackers

NameError: name 'mdx' is not defined

In [16]:
track_var_list = ['AngleMeasured', 'AngleSetpoint']

In [ ]:
# Live value trackers data
track_liv_values = opc.get_live_values_data(track_var_list, trackers)
track_liv_values

In [18]:
# 1 day trackers data
await opc.get_agg_hist_value_data(start_time, end_time, pro_interval, agg_name, trackers, track_var_list)

In [ ]:
# 15 day trackers data 

await opc.get_agg_hist_value_data(start_time, end_time1, pro_interval, agg_name, trackers, track_var_list)

In [ ]:
# 1 month trackers data 
await opc.get_agg_hist_value_data(start_time, end_time2, pro_interval, agg_name, trackers, track_var_list)